In [56]:
import ulmo
import numpy as np 
import pandas as pd
import plotly.express as px
import json
from datetime import datetime as dt, date


In [137]:
#This is the latest CUAHSI API endpoint
wsdlurl = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'

In [26]:
sites = ulmo.cuahsi.wof.get_sites(wsdlurl)


In [205]:
sites_df = pd.DataFrame.from_dict(sites, orient='index').dropna().reset_index()
sites_df = sites_df.rename(columns = {'index':'sitecode'})
sites_df.head()

,sitecode,code,name,network,location,elevation_m,site_property
0,SNOTEL:301_CA_SNTL,301_CA_SNTL,Adin Mtn,SNOTEL,"{'latitude': '41.2358283996582', 'longitude': ...",1886.7120361328125,"{'county': 'Modoc', 'state': 'California', 'si..."
1,SNOTEL:907_UT_SNTL,907_UT_SNTL,Agua Canyon,SNOTEL,"{'latitude': '37.522171020507813', 'longitude'...",2712.719970703125,"{'county': 'Kane', 'state': 'Utah', 'site_comm..."
2,SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,"{'latitude': '45.59722900390625', 'longitude':...",2529.840087890625,"{'county': 'Madison', 'state': 'Montana', 'sit..."
3,SNOTEL:1267_AK_SNTL,1267_AK_SNTL,Alexander Lake,SNOTEL,"{'latitude': '61.749668121337891', 'longitude'...",48.768001556396484,"{'county': 'Matanuska-Susitna', 'state': 'Alas..."
4,SNOTEL:908_WA_SNTL,908_WA_SNTL,Alpine Meadows,SNOTEL,"{'latitude': '47.779571533203125', 'longitude'...",1066.800048828125,"{'county': 'King', 'state': 'Washington', 'sit..."


In [206]:
location = pd.json_normalize(sites_df['location'])
site_property = pd.json_normalize(sites_df['site_property'])

In [208]:
snotel_meta = pd.concat(objs = [sites_df[['network','sitecode','name','elevation_m']], location, site_property], ignore_index=False, axis = 1)
snotel_meta['begin_date'] = snotel_meta['site_comments'].str.split(' ').str[0].str.split('=').str[1]
snotel_meta = snotel_meta[['network','sitecode','name','state','county','elevation_m','latitude','longitude','begin_date']]
snotel_meta = snotel_meta.astype({'begin_date':'datetime64'})
snotel_meta.sample(5)

,network,sitecode,name,state,county,elevation_m,latitude,longitude,begin_date
499,SNOTEL,SNOTEL:612_UT_SNTL,Mammoth-Cottonwood,Utah,Sanpete,2654.503173828125,39.683380126953125,-111.31817626953125,1978-10-01
609,SNOTEL,SNOTEL:1272_NV_SNTL,ONeil Creek,Nevada,Elko,1987.2960205078125,41.864200592041016,-115.08316040039063,2017-09-30
376,SNOTEL,SNOTEL:533_UT_SNTL,Horse Ridge,Utah,Morgan,2499.05517578125,41.313720703125,-111.44624328613281,1978-10-01
41,SNOTEL,SNOTEL:326_WY_SNTL,Beartooth Lake,Wyoming,Park,2852.927978515625,44.943069458007813,-109.56742858886719,1979-10-01
382,SNOTEL,SNOTEL:535_ID_SNTL,Humboldt Gulch,Idaho,Shoshone,1295.4000244140625,47.531780242919922,-115.77642822265625,1978-10-01


In [209]:
# States of interest
states_of_interest = ["Washington",'Oregon','California','Idaho','Montana','Colorado','New Mexico','Utah','Wyoming']


# Counties with many ski resorts
wa_c = ['Whatcom','Chelan','Kittitas']
or_c = ['Deschutes','Clackamas']
ca_c = ['Placer','Nevada','Amador','Mono','San Bernardino']
id_c = ['Bonner','Blaine','Valley']
mt_c = ['Gallatin','Madison','Flathead']
co_c = ['Summit','San Juan','Pitkin','Routt']
nm_c = ['Taos']
ut_c = ['Salt Lake','Utah','Weber']
wy_c = ['Teton','Sublette','Albany']

counties_of_interest = wa_c + or_c + ca_c + id_c + mt_c + co_c + nm_c + ut_c + wy_c

In [210]:
states_of_interest = ["Washington",'Oregon','California','Idaho','Montana','Colorado','New Mexico','Utah']

fig = px.scatter(snotel_meta.query("state in @states_of_interest and county in @counties_of_interest"), x = 'begin_date',y = 'elevation_m', color = 'state')
fig.show()

In [212]:
snotel_meta.query("state in @states_of_interest and county in @counties_of_interest and begin_date < '1981-01-01'").groupby('state')['sitecode'].nunique()

state
California    10
Colorado      15
Idaho         14
Montana       22
New Mexico     2
Oregon         8
Utah          15
Washington     9
Name: sitecode, dtype: int64

In [243]:
snotel_sample = snotel_meta.query("state in @states_of_interest and county in @counties_of_interest and begin_date < '1980-10-01'").groupby('state').sample(frac = .25).sort_values(by = 'state')
snotel_sample

,network,sitecode,name,state,county,elevation_m,latitude,longitude,begin_date
465,SNOTEL,SNOTEL:587_CA_SNTL,Lobdell Lake,California,Mono,2819.09521484375,38.437450408935547,-119.36572265625,1978-10-01
390,SNOTEL,SNOTEL:541_CA_SNTL,Independence Lake,California,Nevada,2541.42236328125,39.427520751953125,-120.31342315673828,1978-10-01
248,SNOTEL,SNOTEL:467_CO_SNTL,Elk River,Colorado,Routt,2651.760009765625,40.847808837890625,-106.96871185302734,1978-10-01
392,SNOTEL,SNOTEL:542_CO_SNTL,Independence Pass,Colorado,Pitkin,3230.8798828125,39.075389862060547,-106.61168670654297,1979-10-01
804,SNOTEL,SNOTEL:802_CO_SNTL,Summit Ranch,Colorado,Summit,2865.1201171875,39.717960357666016,-106.15802001953125,1979-10-01
336,SNOTEL,SNOTEL:505_CO_SNTL,Grizzly Peak,Colorado,Summit,3383.280029296875,39.646308898925781,-105.86972808837891,1979-10-01
385,SNOTEL,SNOTEL:537_ID_SNTL,Hyndman,Idaho,Blaine,2322.575927734375,43.710769653320313,-114.15894317626953,1979-10-01
485,SNOTEL,SNOTEL:601_ID_SNTL,Lost-Wood Divide,Idaho,Blaine,2407.919921875,43.824321746826172,-114.26402282714844,1979-10-01
209,SNOTEL,SNOTEL:439_ID_SNTL,Deadwood Summit,Idaho,Valley,2090.927978515625,44.545139312744141,-115.56379699707031,1978-10-01
717,SNOTEL,SNOTEL:740_ID_SNTL,Secesh Summit,Idaho,Valley,1993.3919677734375,45.188480377197266,-115.97151947021484,1979-10-01


In [248]:
def get_snotel_data(wsdlurl, sitecode, variablecode, start_date, end_date):
    # query the appropriate data 
    data = ulmo.cuahsi.wof.get_values(wsdlurl, sitecode, variablecode, start = start_date, end = end_date)
    # make the data pretty
    df = pd.DataFrame.from_dict(data['values'])[['datetime','value']]
    df = (df
    .assign(sitecode = sitecode)
    .rename(columns = {'value':variablecode})
    .astype({'datetime':'datetime64'})
    .iloc[:,[0,2,1]]
    )
    return(df)

In [249]:
#This is the latest CUAHSI API endpoint
wsdlurl = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'
# Sitecode
sitecode = 'SNOTEL:655_OR_SNTL' 
#Daily SWE
#variablecode = 'SNOTEL:WTEQ_D'
#Daily snow depth
variablecode = 'SNOTEL:WTEQ_sm'

b = get_snotel_data(wsdlurl= wsdlurl, sitecode = sitecode, variablecode=variablecode, start_date= '1980-01-01', end_date = '2022-09-30')

In [261]:
snotel_sample_list = snotel_sample['sitecode'].values.tolist()
swe_df = pd.DataFrame()

for i in snotel_sample_list:
    data = get_snotel_data(wsdlurl= wsdlurl, sitecode = i, variablecode=variablecode, start_date = '1980-10-01', end_date = '2022-09-30')
    swe_df = pd.concat([swe_df,data], axis = 0)

In [263]:
swe = (swe_df
    .rename(columns={'SNOTEL:WTEQ_sm':'swe'})
    .assign(year = lambda x: x.datetime.dt.year,
            doy = lambda x: x.datetime.dt.dayofyear)
)
# Day of Water Year: Sept 30 is last day, October 1 is first day
swe['dowy'] = swe['doy'] - 273
swe.loc[swe['dowy'] <= 0, 'dowy'] += 365
# Add in snotel meta data
swe = swe.merge(snotel_meta, on = 'sitecode', how = 'left')
swe.head()


,datetime,sitecode,swe,year,doy,dowy
0,1980-10-01,SNOTEL:587_CA_SNTL,0.0,1980,275,2
1,1980-10-16,SNOTEL:587_CA_SNTL,0.6,1980,290,17
2,1980-10-31,SNOTEL:587_CA_SNTL,0.0,1980,305,32
3,1980-11-15,SNOTEL:587_CA_SNTL,1.1,1980,320,47
4,1980-11-30,SNOTEL:587_CA_SNTL,0.9,1980,335,62


In [264]:
swe = swe.merge(snotel_meta, on = 'sitecode', how = 'left')
swe.head()

,datetime,sitecode,swe,year,doy,dowy,network,name,state,county,elevation_m,latitude,longitude,begin_date
0,1980-10-01,SNOTEL:587_CA_SNTL,0.0,1980,275,2,SNOTEL,Lobdell Lake,California,Mono,2819.09521484375,38.437450408935547,-119.36572265625,1978-10-01
1,1980-10-16,SNOTEL:587_CA_SNTL,0.6,1980,290,17,SNOTEL,Lobdell Lake,California,Mono,2819.09521484375,38.437450408935547,-119.36572265625,1978-10-01
2,1980-10-31,SNOTEL:587_CA_SNTL,0.0,1980,305,32,SNOTEL,Lobdell Lake,California,Mono,2819.09521484375,38.437450408935547,-119.36572265625,1978-10-01
3,1980-11-15,SNOTEL:587_CA_SNTL,1.1,1980,320,47,SNOTEL,Lobdell Lake,California,Mono,2819.09521484375,38.437450408935547,-119.36572265625,1978-10-01
4,1980-11-30,SNOTEL:587_CA_SNTL,0.9,1980,335,62,SNOTEL,Lobdell Lake,California,Mono,2819.09521484375,38.437450408935547,-119.36572265625,1978-10-01


In [289]:
fig = px.line(swe.query("state =='Colorado'"), x = 'datetime', y = 'swe', color = 'sitecode')
fig.show()

In [280]:
swe.groupby(['name','sitecode','state','county'])['swe'].max()

name                 sitecode            state       county   
Ben Lomond Peak      SNOTEL:332_UT_SNTL  Utah        Weber         9.9
Ben Lomond Trail     SNOTEL:333_UT_SNTL  Utah        Weber         9.9
Blazed Alder         SNOTEL:351_OR_SNTL  Oregon      Clackamas    91.8
Clover Meadow        SNOTEL:403_MT_SNTL  Montana     Madison       9.9
Deadwood Summit      SNOTEL:439_ID_SNTL  Idaho       Valley        9.9
Elk River            SNOTEL:467_CO_SNTL  Colorado    Routt         9.9
Emery Creek          SNOTEL:469_MT_SNTL  Montana     Flathead      9.9
Grizzly Peak         SNOTEL:505_CO_SNTL  Colorado    Summit        9.9
Hyndman              SNOTEL:537_ID_SNTL  Idaho       Blaine        9.9
Independence Lake    SNOTEL:541_CA_SNTL  California  Nevada        9.8
Independence Pass    SNOTEL:542_CO_SNTL  Colorado    Pitkin        9.9
Lily Lake            SNOTEL:579_UT_SNTL  Utah        Summit        9.9
Lobdell Lake         SNOTEL:587_CA_SNTL  California  Mono          9.9
Lost-Wood Divi

In [290]:
snotel = swe.query("name =='Summit Ranch'")

fig = px.line(snotel, 
            x = 'datetime', 
            y = 'swe',
            title = 'Snow Water Equivalent',
            color_discrete_sequence=px.colors.sequential.Inferno)
fig.show()